In [1]:
import pandas as pd
import numpy as np

# ===============================
# 1. 데이터 로드
# ===============================
df = pd.read_csv("data/driving_log.csv")

# ===============================
# 2. 임계값 정의
# ===============================
HARD_BRAKE_THRESHOLD = -3.0   # m/s^2
HARD_ACCEL_THRESHOLD = 3.0    # m/s^2

# ===============================
# 3. 급가감속 이벤트 탐지
# ===============================
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]

hard_brake_count = len(hard_brake_df)
hard_accel_count = len(hard_accel_df)
total_hard_events = hard_brake_count + hard_accel_count

# ===============================
# 4. 주행 시간 / 거리 계산
# ===============================
total_time_sec = df["time_sec"].max() - df["time_sec"].min()

avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# ===============================
# 5. 지표 계산
# ===============================
# events per minute
events_per_min = (
    total_hard_events / (total_time_sec / 60)
    if total_time_sec >= 60 else None
)

# events per km
events_per_km = (
    total_hard_events / distance_km
    if distance_km > 0 else None
)

# hard braking ratio
hard_brake_ratio = (
    hard_brake_count / total_hard_events
    if total_hard_events > 0 else 0
)

# speed variability (표준편차)
speed_variability = df["speed_kmh"].std()

# ===============================
# 6. 점수 정규화 함수
# ===============================
def normalize(value, max_value):
    if value is None:
        return 0
    return min(value / max_value, 1.0)

# ===============================
# 7. 공격성 점수 계산
# ===============================
score = 0

score += normalize(events_per_km, 10) * 30        # km당 이벤트
score += normalize(events_per_min, 5) * 30        # 분당 이벤트
score += hard_brake_ratio * 20                    # 급제동 비율
score += normalize(speed_variability, 20) * 20   # 속도 변동성

aggressiveness_score = round(score, 1)

# ===============================
# 8. 점수 기반 등급 분류
# ===============================
def classify_score(score):
    if score < 30:
        return "Calm Driver"
    elif score < 60:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

driver_grade = classify_score(aggressiveness_score)

# ===============================
# 9. 결과 출력
# ===============================
print("Aggressive Driving Score Result")
print("=" * 40)
print(f"Hard Acceleration Events : {hard_accel_count}")
print(f"Hard Braking Events      : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.2f}")

print("-" * 40)
print(f"Events per Minute        : {events_per_min if events_per_min is not None else 'Insufficient Data'}")
print(f"Events per km            : {events_per_km if events_per_km is not None else 'Insufficient Data'}")
print(f"Hard Brake Ratio         : {hard_brake_ratio:.2f}")
print(f"Speed Variability        : {speed_variability:.2f}")

print("-" * 40)
print(f"Aggressiveness Score     : {aggressiveness_score} / 100")
print(f"Driver Grade             : {driver_grade}")



Aggressive Driving Score Result
Hard Acceleration Events : 3
Hard Braking Events      : 3
Total Hard Events        : 6
Total Time (sec)         : 9.0
Distance (km)            : 0.06
----------------------------------------
Events per Minute        : Insufficient Data
Events per km            : 94.11764705882352
Hard Brake Ratio         : 0.50
Speed Variability        : 19.03
----------------------------------------
Aggressiveness Score     : 59.0 / 100
Driver Grade             : Normal Driver
